In [1]:
#importing necessary things
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import config #you need to make a config.py file in the folder, already in .gitignore so it shouldn't be pushed to repo

In [2]:
#paths to import the csvs
path1 = 'sephora_df.csv'
path2 = 'ulta_df.csv'
sephora = pd.read_csv(path1)
ulta = pd.read_csv(path2)
# print(sephora.head())
# ulta.head()

In [3]:
#this is where you are connecting to the database and are creating the database
con = psycopg2.connect(dbname='postgres',
      user=config.username, host='',
      password=config.password)

con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

cur = con.cursor()
cur.execute('CREATE DATABASE {};'.format('makeup_db'))

In [4]:
#connecting to the database that you just created and starting a session
rds_connection_string = f"{config.username}:{config.password}@localhost:5432/makeup_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
# Confirm tables NOTE: should be empty at first and then when run again after the next few statements the tables should show up
engine.table_names()

['all_makeup', 'sephora', 'ulta']

In [6]:
#putting it all in one big table just in case its easier to use for flask
all_makeup = sephora.merge(ulta, how='outer')
all_makeup.tail()
all_makeup.to_csv('all_makeup.csv')

In [7]:
#adding all makeup to the postgres database
all_makeup.to_sql(name='all_makeup', con=engine, index=False)

In [8]:
#adding sephora to the database
sephora.to_sql(name='sephora', con=engine, index=False)

In [9]:
#adding ulta to the database
ulta.to_sql(name='ulta', con=engine, index=False)